In [3]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score ,GridSearchCV
import joblib
import os

In [4]:
df = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
df = addcolumns(df)
# def combine_weather_10mins(df1:pd.DataFrame):
#     weather_df = pd.read_csv("../資料前處理以及第三方資料/10mins_weather_data.csv")
#     weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
#     weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
#     weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
#     weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
#     weather_df.fillna(0,inplace=True)
#     df = pd.merge(
#         df1,
#         weather_df,
#         left_on=["日期","時間","時段"],
#         right_on=["日期","時間","時段"],
#         how="left",
#         suffixes=["","_r"]
#     )

#     return(df)
# df = combine_weather_10mins(df)

In [65]:

def combine_weather_1hour(df1:pd.DataFrame):
    weather_df = pd.read_csv("../資料前處理以及第三方資料/station466990weather.csv")
    weather_df = weather_df.loc[:,["DateTime","TX01","GR01","RH01","UV01"]]
    weather_df["DateTime"] = weather_df["DateTime"].astype(str)
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].str.slice(0,8))
    weather_df["時間"] = weather_df["DateTime"].str.slice(8,10)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間"],
        right_on=["日期","時間"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
df = combine_weather_1hour(df)

In [5]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df["LocationCode"] = df["LocationCode"].astype(str).str.zfill(2)
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
df = selecttime(df)

In [52]:
df

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude,DateTime_r,TX01,GR01,RH01,UV01
15,20240101090001,01,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,...,64.0,7-10,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
16,20240101091001,01,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
17,20240101092001,01,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
18,20240101093001,01,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,...,65.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
19,20240101094001,01,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10683,20240724162001,01,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,...,93.0,21-30,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
10684,20240724163001,01,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,...,89.0,16-20,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
10685,20240724164001,01,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,...,87.0,3-6,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
10686,20240724165001,01,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,...,88.0,3-6,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22


In [ ]:
def combine_sundf(df:pd.DataFrame):
    sun_df = pd.read_csv("../太陽仰角與日射量資料計算整理/sunlight_data.csv")
    sun_df["location"] = sun_df["location"].astype(str).str.zfill(2)
    sun_df["時間"] = pd.to_datetime(sun_df["時間"])
    ["太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"]
    df = df.merge(
        sun_df.loc[:,["location","時間","太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"]],
        left_on=["TimeGroup","LocationCode"],
        right_on=["時間","location"],
        suffixes=["","_r"]
        how="left"
    )
    return(df)
combine_sundf(df).columns

Index(['Serial', 'LocationCode', 'TimeGroup', 'WindSpeed(m/s)',
       'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)',
       'Power(mW)', '日期', '時間_x', '時段', '位置', 'StationId', 'DateTime',
       'SunshineDuration', 'UVIndex', 'AirTemperature',
       'DailyExtremeHighAirTemperature', 'DailyExtremeLowAirTemperature',
       'RelativeHumidity', 'VisibilityDescription', 'StationLatitude',
       'StationLongitude', 'StationAltitude', 'DateTime_r', 'TX01', 'GR01',
       'RH01', 'UV01', 'location', '時間_y', '太陽仰角', '日射量 (W/m²)', '太陽方位角',
       '太陽能板日射量 (W/m²)'],
      dtype='object')

In [6]:
weather_df = pd.read_csv(f"../資料前處理以及第三方資料/weather_sun_combine.csv")
weather_df["日期"] = pd.to_datetime(weather_df["日期"])
weather_df["location"] = weather_df["location"].astype(str).str.zfill(2)
weather_df["時間"] = weather_df["時間"].astype(str).str.zfill(2)
weather_df["時段"] = weather_df["時段"].astype(str).str.zfill(2)
weather_df

,location,DateTime,太陽仰角,日射量 (W/m²),太陽方位角,太陽能板日射量 (W/m²),StationId,SunshineDuration,UVIndex,AirTemperature,...,StationLongitude,StationAltitude,日期,時間,時段,DateTime_r,TX01,GR01,RH01,UV01
0,01,2024-01-01 00:00:00,-88.366208,0.0,138.515569,0.0,466990.0,0.9,0.0,19.2,...,121.61327,16.1,2024-01-01,00,00,2024010124,17.1,0.0,79,0.0
1,01,2024-01-01 00:10:00,-86.202505,0.0,137.831949,0.0,466990.0,0.0,0.0,19.2,...,121.61327,16.1,2024-01-01,00,10,2024010124,17.1,0.0,79,0.0
2,01,2024-01-01 00:20:00,-83.943446,0.0,137.158723,0.0,466990.0,0.0,0.0,19.2,...,121.61327,16.1,2024-01-01,00,20,2024010124,17.1,0.0,79,0.0
3,01,2024-01-01 00:30:00,-81.666617,0.0,136.493326,0.0,466990.0,0.0,0.0,19.2,...,121.61327,16.1,2024-01-01,00,30,2024010124,17.1,0.0,79,0.0
4,01,2024-01-01 00:40:00,-79.383910,0.0,135.833303,0.0,466990.0,0.0,0.0,19.2,...,121.61327,16.1,2024-01-01,00,40,2024010124,17.1,0.0,79,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746635,17,2024-10-31 23:10:00,-75.947259,0.0,142.108191,0.0,466990.0,0.0,0.0,25.6,...,121.61327,16.1,2024-10-31,23,10,2024103123,25.6,0.0,94,0.0
746636,17,2024-10-31 23:20:00,-77.567816,0.0,141.327489,0.0,466990.0,0.0,0.0,25.5,...,121.61327,16.1,2024-10-31,23,20,2024103123,25.6,0.0,94,0.0
746637,17,2024-10-31 23:30:00,-78.921182,0.0,140.572822,0.0,466990.0,0.0,0.0,25.7,...,121.61327,16.1,2024-10-31,23,30,2024103123,25.6,0.0,94,0.0
746638,17,2024-10-31 23:40:00,-79.899069,0.0,139.840382,0.0,466990.0,0.0,0.0,25.6,...,121.61327,16.1,2024-10-31,23,40,2024103123,25.6,0.0,94,0.0


In [7]:
i = 1
location = "0" + str(i) if i<10 else str(i)
df = pd.read_csv(f"./train_combine_groupby/{location}_traingroupby.csv")
df = addcolumns(df)
df = selecttime(df)
df

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,時間,時段,位置
15,20240101090001,01,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,09,00,01
16,20240101091001,01,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,09,10,01
17,20240101092001,01,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,09,20,01
18,20240101093001,01,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,09,30,01
19,20240101094001,01,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,09,40,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,20240724162001,01,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,16,20,01
10613,20240724163001,01,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,16,30,01
10614,20240724164001,01,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,16,40,01
10615,20240724165001,01,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,16,50,01


In [133]:
df = df.merge(
    weather_df.loc[weather_df["location"]==location,],
    left_on=["日期","時間",	"時段"],
    right_on=["日期","時間","時段"],
)
df

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude,DateTime_r,TX01,GR01,RH01,UV01
0,20240101090001,01,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,...,64.0,7-10,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
1,20240101091001,01,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
2,20240101092001,01,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
3,20240101093001,01,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,...,65.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
4,20240101094001,01,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,...,64.0,21-30,23.975128,121.61327,16.1,2024010109,18.8,0.19,64,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6370,20240724162001,01,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,...,93.0,21-30,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
6371,20240724163001,01,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,...,89.0,16-20,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
6372,20240724164001,01,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,...,87.0,3-6,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22
6373,20240724165001,01,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,...,88.0,3-6,23.975128,121.61327,16.1,2024072416,25.6,0.07,95,0.22


In [ ]:
d = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"./train_combine_groupby/{location}_traingroupby.csv")
    
    df = addcolumns(df)
    # df = combine_weather_10mins(df)
    # df = combine_weather_1hour(df)
    df = selecttime(df)
    # df = combine_sundf(df)
    df = df.merge(
        weather_df.loc[weather_df["location"]==location,],
        left_on=["日期","時間",	"時段"],
        right_on=["日期","時間","時段"],
    )
    select_cols = [ "TX01","GR01","RH01","UV01",
                    "AirTemperature","RelativeHumidity","SunshineDuration","UVIndex",
                    "太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"
                    ]
    
    X = df.loc[:,select_cols].reset_index(drop=True)
    Y = df.loc[:,["Power(mW)"]].reset_index(drop=True)
    Y = Y.values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

    model = RandomForestRegressor(n_jobs=14)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
    }
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,
        n_jobs=14,
    )
    grid_search.fit(X_train, y_train)
      # 儲存最佳模型和 MSE
    best_model = grid_search.best_estimator_
    scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    MAE = -sum(scores)
    best_model.fit(X_train, y_train)
    
    model_file_name = f'{location}_weather.pkl'  
    model_folder_name = "train_weatherdata_all"         #將模型相關資訊儲存在這個檔案下
    model_file_path = os.path.join("./model_each_location",model_folder_name,model_file_name)
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(best_model, model_file_path)
    print(model_file_name ," MAE:",MAE )
    print(f"模型已保存至: {model_file_path}")
    if model_file_name not in d:
        d[location] = MAE
    

01_weather.pkl  MAE: 539.391397080625
模型已保存至: ./model_each_location\train_weatherdata_all\01_weather.pkl
02_weather.pkl  MAE: 606.0436193086282
模型已保存至: ./model_each_location\train_weatherdata_all\02_weather.pkl
03_weather.pkl  MAE: 699.0763764108516
模型已保存至: ./model_each_location\train_weatherdata_all\03_weather.pkl
04_weather.pkl  MAE: 552.3853501433053
模型已保存至: ./model_each_location\train_weatherdata_all\04_weather.pkl
05_weather.pkl  MAE: 714.0791650409326
模型已保存至: ./model_each_location\train_weatherdata_all\05_weather.pkl
06_weather.pkl  MAE: 773.4739943843185
模型已保存至: ./model_each_location\train_weatherdata_all\06_weather.pkl
07_weather.pkl  MAE: 672.6760400321996
模型已保存至: ./model_each_location\train_weatherdata_all\07_weather.pkl
08_weather.pkl  MAE: 448.77019475474185
模型已保存至: ./model_each_location\train_weatherdata_all\08_weather.pkl
09_weather.pkl  MAE: 344.59574351599224
模型已保存至: ./model_each_location\train_weatherdata_all\09_weather.pkl
10_weather.pkl  MAE: 518.2082792991831
模型已保存至

In [ ]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"./model_each_location/{model_folder_name}/d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)

In [ ]:
with open(f"./model_each_location/{model_folder_name}/d.json", "r", encoding="utf-8") as file:
    mae_data = json.load(file)

In [ ]:
#將不同模型load近來並放在字典裡
model_dict = {}
for k1 in mae_data.keys():
    model_file_path = os.path.join(f"./model_each_location/{model_folder_name}/{k1}_weather.pkl")
    model_dict[k1] = joblib.load(model_file_path)

In [89]:
#將不同觀測站的資料load近來並放在字典裡
df_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"./train_combine_groupby/{location}_traingroupby.csv")
    df = addcolumns(df)
    df = combine_weather_10mins(df)
    df = combine_weather_1hour(df)
    df = selecttime(df)
    df = combine_sundf(df)
    df_dict[location] = df

In [ ]:
weather_df = pd.read_csv("../資料前處理以及第三方資料/10mins_weather_data.csv")

In [ ]:
#將檔案讀進來  新增日期時間位置等欄位以便我們找資料對應
DataName = "./36_TestSet_SubmissionTemplate/upload(no answer).csv"
ans = pd.read_csv(DataName, encoding='utf-8')
ans["序號"] = ans["序號"].astype(str)
ans["日期"] = pd.to_datetime(ans["序號"].str.slice(0,8))
ans["時間"] =ans["序號"].str.slice(8,10)
ans["時段"] =ans["序號"].str.slice(10,12)
ans["位置"] =ans["序號"].str.slice(12,14)
ans["模型順位"] = None
ans["使用模型MAE"] = None
select_cols = [ "TX01","GR01","RH01","UV01",
                "AirTemperature","RelativeHumidity","SunshineDuration","UVIndex",
                "太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"
                ]
for i in range(len(ans)):
    date,time,timecount,ans_location = ans.loc[i,"日期"],ans.loc[i,"時間"],ans.loc[i,"時段"],ans.loc[i,"位置"]  #找出答案該個row的時期時間時段以及位置
    mae_table = mae_data[ans_location]   #找出這個位置的所有模型(來自另外16個模型) 並且已按照MAE順序排
    df_sametimedata = df_dict[ans_location]
    search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
    if len(search_df)==1:  #如果有找資料 就用這個模型predict 並且break迴圈  沒有就繼續往MAE較大的模型找 找到都沒有則答案欄位就保持NA
        X_test = search_df.loc[:,select_cols]
        model = model_dict[ans_location]
        predictions = model.predict(X_test)
        ans.loc[i,"答案"] = predictions[0]
    if i %100==0:
        print(i)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [98]:
search_df

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,TX01,GR01,RH01,UV01,location,時間_y,太陽仰角,日射量 (W/m²),太陽方位角,太陽能板日射量 (W/m²)


In [97]:
ans.loc[ans["答案"].isna(),]

,序號,答案,日期,時間,時段,位置,模型順位,使用模型MAE
0,20240117090001,NaN,2024-01-17,09,00,01,None,None
1,20240117091001,NaN,2024-01-17,09,10,01,None,None
2,20240117092001,NaN,2024-01-17,09,20,01,None,None
3,20240117093001,NaN,2024-01-17,09,30,01,None,None
4,20240117094001,NaN,2024-01-17,09,40,01,None,None
...,...,...,...,...,...,...,...,...
9595,20241002161012,NaN,2024-10-02,16,10,12,None,None
9596,20241002162012,NaN,2024-10-02,16,20,12,None,None
9597,20241002163012,NaN,2024-10-02,16,30,12,None,None
9598,20241002164012,NaN,2024-10-02,16,40,12,None,None
